# Import libraries

In [1]:
import pandas_ti
import pandas as pd
import yfinance as yf
import mplfinance as mpf

Check if the decorator works as intended

In [2]:
from pandas_ti.registry import registry_funcs_dict, registry_names_dict
registry_names_dict

{'dataframe': {'Indicator': ['RTR', 'ARTR', 'TR', 'ATR', 'SRTR', 'ZigZag'],
  'Full Name': ['Relative True Range',
   'Average Relative True Range',
   'True Range',
   'Average True Range',
   'Standardized Relative True Range',
   'ZigZag']},
 'series': {'Indicator': ['EMA', 'SMA'],
  'Full Name': ['Exponential Moving Average', 'Simple Moving Average']}}

---

## Downlaod Test Data

In [3]:
ticker = 'AAPL'
df = yf.Ticker(ticker).history(period='1y', rounding=True)[['Open', 'High', 'Low', 'Close', 'Volume']]
df

,Open,High,Low,Close,Volume
Date,,,,,
2024-11-04 00:00:00-05:00,219.97,221.76,218.69,220.98,44944500
2024-11-05 00:00:00-05:00,220.77,222.91,220.12,222.41,28111300
2024-11-06 00:00:00-05:00,221.58,225.02,220.17,221.69,54561100
2024-11-07 00:00:00-05:00,223.59,226.82,223.53,226.43,42137700
2024-11-08 00:00:00-05:00,226.37,227.85,225.61,226.16,38328800
...,...,...,...,...,...
2025-10-28 00:00:00-04:00,268.99,269.89,268.15,269.00,41534800
2025-10-29 00:00:00-04:00,269.28,271.41,267.11,269.70,51086700
2025-10-30 00:00:00-04:00,271.99,274.14,268.48,271.40,69886500


---

## Dataframe Accessor

### Indicators

In [4]:
df.ti.indicators()

,Indicator,Full Name
0,RTR,Relative True Range
1,ARTR,Average Relative True Range
2,TR,True Range
3,ATR,Average True Range
4,SRTR,Standardized Relative True Range
5,ZigZag,ZigZag


In [5]:
df.ti.help('ZigZag')

╭─────────────────────────────────────────── Documentation for ZigZag ────────────────────────────────────────────╮
│                                                                                                                 │
│ ZigZag Indicator                                                                                                │
│                                                                                                                 │
│ Identifies significant price reversals by detecting swing highs and lows,                                       │
│ confirming pivots only when price moves by at least the specified percentage                                    │
│ in the opposite direction.                                                                                      │
│                                                                                                                 │
│ Required Columns                                                                                                │
│ ----------------                                                                                                │
│ High : pd.Series                                                                                                │
│     Series of high prices (Automatically injected).                                                             │
│ Low : pd.Series                                                                                                 │
│     Series of low prices (Automatically injected).                                                              │
│                                                                                                                 │
│ Parameters                                                                                                      │
│ ----------                                                                                                      │
│ pct : float                                                                                                     │
│     Minimum percentage change required to confirm a reversal pivot.                                             │
│     Must be greater than 0. For example, 0.05 means a 5% price movement.                                        │
│                                                                                                                 │
│ Returns                                                                                                         │
│ -------                                                                                                         │
│ ZigZag : pd.Series                                                                                              │
│     Series containing the confirmed pivot points. Non-pivot candles have NaN values.                            │
│     Pivot candles contain the price level (High or Low) at which the pivot occurred.                            │
│                                                                                                                 │
│ Examples                                                                                                        │
│ --------                                                                                                        │
│ >>> df['ZZ'] = df.ti.ZigZag(pct=0.05)  # Simple 5% ZigZag                                                       │
│ >>> # Recommended: smooth High and Low first                                                                    │
│ >>> df['SMA_High_3'] = df['High'].ti.SMA(n=3)                                                                   │
│ >>> df['SMA_Low_3'] = df['Low'].ti.SMA(n=3)                                                                     │
│ >>> df['ZZ_Smooth'] = df.ti.ZigZag(High=df['SMA_High_3'], Low=df['SMA_Low_3'], pct=0.05)                        │
│ >>> df['ZZ_Smooth'].dropna()                          

In [ ]:
df['zigzag'] = df.ti.ZigZag(pct=0.05)
df

,Open,High,Low,Close,Volume,zigzag
Date,,,,,,
2024-11-04 00:00:00-05:00,219.97,221.76,218.69,220.98,44944500,218.69
2024-11-05 00:00:00-05:00,220.77,222.91,220.12,222.41,28111300,NaN
2024-11-06 00:00:00-05:00,221.58,225.02,220.17,221.69,54561100,NaN
2024-11-07 00:00:00-05:00,223.59,226.82,223.53,226.43,42137700,NaN
2024-11-08 00:00:00-05:00,226.37,227.85,225.61,226.16,38328800,NaN
...,...,...,...,...,...,...
2025-10-28 00:00:00-04:00,268.99,269.89,268.15,269.00,41534800,NaN
2025-10-29 00:00:00-04:00,269.28,271.41,267.11,269.70,51086700,NaN
2025-10-30 00:00:00-04:00,271.99,274.14,268.48,271.40,69886500,NaN


In [7]:
from pandas_ti.indicators_dataframe.ZigZag import ZigZagClass
zz = ZigZagClass(pct=0.05)
for idx, row in df.iterrows():
    zz.update(high=row['High'], low=row['Low'], idx=idx)
pivots = zz.pivots()